In [41]:
import json
import pandas as pd
from collections import Counter

In [55]:
def read_json_lines(file_name):
    lines = []
    with open(file_name) as file_in:
        for line in file_in:
            lines.append(json.loads(line))
    return lines

def write_json_lines(file_name,dict_data):
    json_string = json.dumps(dict_data)
    with open(file_name, 'a') as f:
        f.write(json_string+"\n")
        
def get_train_res(file_name):
    all_res = []
#     impo_hp = ['b_size', 'dataset', 'emb_dim', 'lr', 'margin', 'model_name', 'n_neg']
    res = read_json_lines(file_name)
    for i in res:
        result = i['train_params']
#         result = {hp:result[hp] for hp in impo_hp}
        result.update(i['test_results'])
        all_res.append(result)
    return pd.DataFrame(all_res)

def get_best_model():
    part1_experiments = get_train_res('edukg_experiments_part1.jsonl')
    part2_experiments = get_train_res('edukg_experiments_part2.jsonl')
    experiments = pd.concat([part1_experiments, part2_experiments])
    best_model  = experiments[experiments['filt_mrr']==experiments['filt_mrr'].max()].to_dict('records')[0]
    return best_model

def get_best_model_results():
    part1_experiments = get_train_res('edukg_experiments_part1.jsonl')
    part2_experiments = get_train_res('edukg_experiments_part2.jsonl')
    experiments = pd.concat([part1_experiments, part2_experiments])
    best_experiments = experiments.sort_values(['filt_mrr'],ascending=False).groupby('model_name').first()
    best_experiments = best_experiments[['filt_hit_1', 'filt_hit_3', 'filt_hit_10',
                                         'filt_mrr', 'b_size', 'lr', 'emb_dim', 'margin', 'n_neg', 's_rep_ratio', 'dropout']]
    
    return best_experiments


## Best Models

In [36]:
get_best_model()

{'n_epochs': 1000,
 'data_home': '/home/vortex/Desktop/Projects/TempTorchKGE_V2/datasets',
 'model_save_name': 'TDistMultModel.bt',
 'loss_name': 'MarginLoss',
 'model_path': 'best_models',
 'time_mode': 'seq',
 'dissimilarity_type': 'L2',
 'model_name': 'TDistMultModel',
 'dataset': 'edukg',
 'n_entities': 13346,
 'n_relations': 12,
 'tem_total': 2012,
 'b_size': 4000,
 'lr': 0.001,
 'emb_dim': 50,
 'margin': 1,
 'n_neg': 10,
 'hit_1': 0.20081131905317307,
 'filt_hit_1': 0.6177013516426086,
 'hit_2': 0.3217346668243408,
 'filt_hit_2': 0.7723654210567474,
 'hit_3': 0.39383846521377563,
 'filt_hit_3': 0.8453880846500397,
 'hit_4': 0.4369121789932251,
 'filt_hit_4': 0.8862268626689911,
 'hit_5': 0.4648910164833069,
 'filt_hit_5': 0.9105423390865326,
 'hit_6': 0.48371344804763794,
 'filt_hit_6': 0.9259060025215149,
 'hit_7': 0.49583107233047485,
 'filt_hit_7': 0.9354596734046936,
 'hit_8': 0.5044337660074234,
 'filt_hit_8': 0.9419397711753845,
 'hit_9': 0.5115117132663727,
 'filt_hit_9': 

In [56]:
xx = get_best_model_results()

In [57]:
xx

,filt_hit_1,filt_hit_3,filt_hit_10,filt_mrr,b_size,lr,emb_dim,margin,n_neg,s_rep_ratio,dropout
model_name,,,,,,,,,,,
DEDistMultModel,0.548242,0.705422,0.913760,0.657922,2000,0.0010,100,1,10,0.1,0.2
DETransEModel,0.484921,0.583002,0.714077,0.560718,4000,0.0001,100,1,40,0.1,0.2
TDistMultModel,0.617701,0.845388,0.950847,0.742037,4000,0.0010,50,1,10,NaN,NaN
TTransEModel,0.612630,0.795618,0.900832,0.716565,4000,0.0010,50,20,30,NaN,NaN


In [64]:
print(xx[['filt_hit_1', 'filt_hit_3', 'filt_hit_10', 'filt_mrr']].reset_index().to_latex(index=False))

\begin{tabular}{lrrrr}
\toprule
     model\_name &  filt\_hit\_1 &  filt\_hit\_3 &  filt\_hit\_10 &  filt\_mrr \\
\midrule
DEDistMultModel &    0.548242 &    0.705422 &     0.913760 &  0.657922 \\
  DETransEModel &    0.484921 &    0.583002 &     0.714077 &  0.560718 \\
 TDistMultModel &    0.617701 &    0.845388 &     0.950847 &  0.742037 \\
   TTransEModel &    0.612630 &    0.795618 &     0.900832 &  0.716565 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3840209/1181275353.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(xx[['filt_hit_1', 'filt_hit_3', 'filt_hit_10', 'filt_mrr']].reset_index().to_latex(index=False))


In [65]:
print(xx[['lr', 'emb_dim', 'margin', 'n_neg', 's_rep_ratio', 'dropout']].reset_index().to_latex(index=False))

\begin{tabular}{lrrrrrr}
\toprule
     model\_name &     lr &  emb\_dim &  margin &  n\_neg &  s\_rep\_ratio &  dropout \\
\midrule
DEDistMultModel & 0.0010 &      100 &       1 &     10 &          0.1 &      0.2 \\
  DETransEModel & 0.0001 &      100 &       1 &     40 &          0.1 &      0.2 \\
 TDistMultModel & 0.0010 &       50 &       1 &     10 &          NaN &      NaN \\
   TTransEModel & 0.0010 &       50 &      20 &     30 &          NaN &      NaN \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3840209/3065102251.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(xx[['lr', 'emb_dim', 'margin', 'n_neg', 's_rep_ratio', 'dropout']].reset_index().to_latex(index=False))
